# Data section

Build classifiers and retrieve prediction

## Set up

In [1]:
import math
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Read the data. Make sure that there is a column called "Noshow", with the indicators of no-shows (1 vs 0)

In [35]:
#import statsmodels.api as sm
df = pd.read_csv('data.csv',sep=',')

The no-show rate

In [36]:
df['Noshow'].mean()

0.341

In [37]:
X = df.drop(columns=['Noshow'],axis=1) 

In [38]:
Y = df['Noshow']

In [39]:
Y.mean()

0.341

# Split data into training and testing

In [40]:
from sklearn.model_selection import train_test_split
test_size_proportion = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size_proportion, random_state=0)

# Find typical variance obtained by probabilistic classifiers

Study the distribution of the probabilities obtained by probabilistic classifiers

The first thing to do it to obtain a vector of probabilities. We execute a cross-validation and label each instance with the predicted probability obtained when it was out of sample

In [41]:
import sklearn as sk
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

iter = 1
nfolds = 10
from sklearn.metrics import roc_curve, auc
X_copy = X_train.copy()


clfs = [sk.ensemble.RandomForestClassifier(n_jobs=-1,random_state=0), sk.naive_bayes.GaussianNB(),
        sk.linear_model.LogisticRegression(n_jobs=-1),
        sk.ensemble.AdaBoostClassifier(random_state=0),\
        MLPClassifier(random_state=0)]
#clfs = [sk.ensemble.AdaBoostClassifier(random_state=0)]
#clfs=[sk.linear_model.LogisticRegression(n_jobs=-1)]
class_df = pd.DataFrame()
classifiers_by_name = {}
for cl in clfs:
    kf = KFold(n_splits=nfolds,random_state=0,shuffle=True)
    clName = str(cl)[:15] 
    classifiers_by_name[clName] = cl
    class_df.loc[clName,'auc'] = 0
    # compute the cross-validated auc
    print(f'Testing {clName}: ')
    for tr_ind,te_ind in kf.split(X_train,Y_train):
        
        tr_X = X_train.iloc[tr_ind]
        tr_Y = Y_train.iloc[tr_ind]
        te_X = X_train.iloc[te_ind]
        te_Y = Y_train.iloc[te_ind]
        print(f'Training in progress for iteration {iter}')
        iter+=1
        cl.fit(tr_X,tr_Y)
        pred_scores = cl.predict_proba(te_X)[:,1]
        X_copy.loc[X_copy.iloc[te_ind].index,'pred_scores_' + (clName)] = pred_scores
        #X_copy.iloc[te_ind].loc[:,'pred_scores_' + (clName)] = pred_scores
        
        # Compute ROC curve and ROC area for each class
        fpr, tpr, _ = roc_curve(te_Y, pred_scores)
        class_df.loc[clName,'auc'] += auc(fpr, tpr)
    class_df.loc[clName,'auc'] /= nfolds

Testing RandomForestCla: 
Training in progress for iteration 1


c:\program files\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training in progress for iteration 2
Training in progress for iteration 3
Training in progress for iteration 4
Training in progress for iteration 5
Training in progress for iteration 6
Training in progress for iteration 7
Training in progress for iteration 8
Training in progress for iteration 9
Training in progress for iteration 10
Testing GaussianNB(prio: 
Training in progress for iteration 11
Training in progress for iteration 12
Training in progress for iteration 13
Training in progress for iteration 14
Training in progress for iteration 15
Training in progress for iteration 16
Training in progress for iteration 17
Training in progress for iteration 18
Training in progress for iteration 19
Training in progress for iteration 20
Testing LogisticRegress: 
Training in progress for iteration 21
Training in progress for iteration 22
Training in progress for iteration 23
Training in progress for iteration 24
Training in progress for iteration 25
Training in progress for iteration 26
Traini

c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default s

Training in progress for iteration 33
Training in progress for iteration 34
Training in progress for iteration 35
Training in progress for iteration 36
Training in progress for iteration 37
Training in progress for iteration 38
Training in progress for iteration 39
Training in progress for iteration 40
Testing MLPClassifier(a: 
Training in progress for iteration 41
Training in progress for iteration 42
Training in progress for iteration 43
Training in progress for iteration 44
Training in progress for iteration 45
Training in progress for iteration 46
Training in progress for iteration 47
Training in progress for iteration 48
Training in progress for iteration 49
Training in progress for iteration 50


Add columns "pred_probas_"+classifiername 

In [42]:
for clname in class_df.index:
    if clname.startswith('LogisticReg'):
        X_copy['pred_probas_'+clname] = X_copy['pred_scores_' + clname]
    else:
        logreg = sk.linear_model.LogisticRegression(n_jobs=-1)
        logreg.fit(X_copy[['pred_scores_' + clname]],Y_train)
        X_copy['pred_probas_'+clname] = logreg.predict_proba(X_copy[['pred_scores_' + clname]])[:,1]

c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default s

Find which classifier has the smallest brier score

In [43]:
from sklearn.metrics import brier_score_loss

best_classifier_name = ''
lowest_brier_loss = 10000
for clname in class_df.index:
    score = brier_score_loss(Y_train,X_copy['pred_probas_'+clname])
    print(f'{clname} obtains a brier score of {score}')
    class_df.loc[clname,'brier'] = score
    if score < lowest_brier_loss:
        lowest_brier_loss = score
        best_classifier_name = clname

RandomForestCla obtains a brier score of 0.19988208304875982
GaussianNB(prio obtains a brier score of 0.17579349944421907
LogisticRegress obtains a brier score of 0.17521089770346784
AdaBoostClassif obtains a brier score of 0.22090190210029903
MLPClassifier(a obtains a brier score of 0.19312831007088035


In [44]:
class_df

,auc,brier
RandomForestCla,0.692319,0.199882
GaussianNB(prio,0.777705,0.175793
LogisticRegress,0.778963,0.175211
AdaBoostClassif,0.744313,0.220902
MLPClassifier(a,0.731893,0.193128


# The best classifier is...

In [45]:
best_classifier_name

'LogisticRegress'

# Calculate predicted probabilities on test set

Now we have the brier losses over the training set. Train on the whole training set and label the test set

In [46]:
best_classifier = classifiers_by_name[best_classifier_name]
best_classifier.fit(X_train,Y_train)
pred_scores = pd.DataFrame(index=X_test.index, 
        data = best_classifier.predict_proba(X_test)[:,1],
        columns = ['score']
           )
X_train_2 = X_train.copy()
X_train_2['score'] = best_classifier.predict_proba(X_train_2)[:,1]
if best_classifier_name.startswith('LogisticReg'):
    pred_probas = pred_scores
    data=pred_probas['score'].sort_values().values
else:
    logreg = sk.linear_model.LogisticRegression(n_jobs=-1)
    logreg.fit(X_train_2[['score']],Y_train)

    pred_probas = logreg.predict_proba(pred_scores)[:,1]
    data = pred_probas

c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\program files\python36\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Let's analyze the no-show probabilities of the best classifier

AUC on test set

In [47]:
import sklearn
sklearn.metrics.roc_auc_score(Y_test, pred_scores)

0.8371777476255088

Brier's score on the test set

In [48]:
brier_score_loss(Y_test,pred_probas)

0.1526462560960884

In [49]:
X_test['Y_pred'] = pred_probas#.score
X_test['Y_actual'] = Y_test

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\program files\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Write to file

In [50]:
X_test.to_csv('test_set_with_predictions.csv')